# Análisis de datos de nosepoke

## 0. Importación de librerías

In [1]:
import pandas as pd
import os
import docx2txt
from datetime import datetime, timedelta
import re

## 0.1.Funciones auxiliares

In [3]:
path="D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/"

pathMachos=os.path.join(path,'MACHOS')
pathHembras=os.path.join(path,'HEMBRAS')

pathProcesarDatos="D:/jmriv/Documents/GitHub/NosePoke/ProcesarDatos"


In [4]:
def separarLinea(linea:str):
  """
  Cada línea de los archivos tiene una estructura "HH:MM:SS.MS -> DATOS".
  En caso de que no haya una entrada de tiempo se pone como hora '-1'
  
  Parameters:
  linea (String): linea de texto extraída del archivo de datos
  etiqueta (String): indica si hay más de un registro en el archivo "SPLIT" o si no tiene el tiempo "TIEMPO"

  Returns:
  hora, registro (String, String): hora y datos por separado.
  """
  if len(linea.split('->'))==2:
    hora=linea.split('->')[0]
    registro=linea.split('->')[1]
  else:
    hora=-1
    registro=linea
  return hora, registro

In [5]:
def leerTexto(path):
  text=docx2txt.process(path)
  if len(text)==0:
        print(f'Archivo vacío: {path}')
  # Los archivos donde los datos están explícitos no tienen '--------------------------------------------------'
  esCSV=text.find('--------------------------------------------------') < 0
  if 'Perseveracion' in text:
      print(f'{path} El archivo tiene perseveraciones en vez de Tiempo en respuestas impulsivas')
  text=text.split('\n')
  # Se eliminan líneas vacías
  text= list(filter(None, text))
  return text, esCSV

In [6]:
def quitarEncabezadoNoCSV(textArray):
  for index,line in enumerate(textArray):
    if line.find('Van a empezar los 50 ensayos') != -1 or line.find('Ensayo numero') != -1:
      return textArray[index:]
  return []

In [7]:
def extraerValorNumerico(texto):
  return float(re.findall(r'\d+\.?\d*', texto)[0])

In [89]:
def promedioInverso(promedioActual, promedioAnterior, n, tipoPrueba, esAcierto):
  if tipoPrueba == 'HAB':
    if n == 1:
      return promedioActual
    return n*promedioActual - promedioAnterior*(n-1)
  elif tipoPrueba.find('SM') > -1:
    if not esAcierto:
      return (promedioActual - promedioAnterior)*n
    else:
      return n*promedioActual - promedioAnterior*(n-1)

In [68]:
promedioInverso(20798, 16993, 1, 'SM', False)

3805

### Procesar ensayo

In [75]:
def procesarEnsayo(ensayoActual, esCSV, ultimoEnsayo, tipoPrueba, metioNariz):
  if tipoPrueba == 'HAB':
    if esCSV:
      aciertos=extraerValorNumerico(ensayoActual[1])
      tiempoTotalImpulsivas=extraerValorNumerico(ensayoActual[3])
      omisiones=extraerValorNumerico(ensayoActual[4])
      categorias=extraerValorNumerico(ensayoActual[6])
      correctasDer=extraerValorNumerico(ensayoActual[7])
      latenciaPromedio=extraerValorNumerico(ensayoActual[9])
      maxAciertosSeguidos=extraerValorNumerico(ensayoActual[10])
    else:
      aciertos=extraerValorNumerico(ensayoActual[0])
      tiempoTotalImpulsivas=extraerValorNumerico(ensayoActual[2])
      omisiones=extraerValorNumerico(ensayoActual[3])
      categorias=extraerValorNumerico(ensayoActual[5])
      correctasDer=extraerValorNumerico(ensayoActual[6])
      latenciaPromedio=extraerValorNumerico(ensayoActual[8])
      maxAciertosSeguidos=extraerValorNumerico(ensayoActual[9])

    n = aciertos+omisiones
    esAcierto = aciertos != ultimoEnsayo['Aciertos acomulados']

    if not esAcierto:
      latencia=""
    else:
      latencia = promedioInverso(latenciaPromedio, ultimoEnsayo['Latencia promedio'], aciertos,tipoPrueba,esAcierto)
      if metioNariz >= 0 and abs(metioNariz - latencia) <= 50:
        latencia = metioNariz
      elif (n - ultimoEnsayo["No. ensayo"]) != 1:
        latencia=""
        # promedioInverso(latenciaPromedio, ultimoEnsayo['Latencia promedio'], aciertos) if esAcierto else 0

    nuevoEnsayo = {
        "No. ensayo": n,
        "Fue acierto": esAcierto,
        "Fue acierto derecho": correctasDer != ultimoEnsayo['Total correctas derecha'] if esAcierto else '',
        "%Aciertos": aciertos/n,
        "%Omisiones":omisiones/n,
        "Latencia de respuesta": latencia,
        "Tiempo en respuestas impulsivas": tiempoTotalImpulsivas - ultimoEnsayo['Tiempo total impulsivas'],
        "Categorías":categorias,
        "Máximo aciertos seguidos":maxAciertosSeguidos,
        "Aciertos acomulados":aciertos,
        "Tiempo total impulsivas":tiempoTotalImpulsivas,
        "Total correctas derecha":correctasDer,
        "Latencia promedio":latenciaPromedio
        }
  elif tipoPrueba.find('SM')>-1:
    if esCSV:
      aciertos=extraerValorNumerico(ensayoActual[1])
      tiempoTotalImpulsivas=extraerValorNumerico(ensayoActual[3])
      omisiones=extraerValorNumerico(ensayoActual[4])
      categorias=extraerValorNumerico(ensayoActual[6])
      latenciaPromedio=extraerValorNumerico(ensayoActual[7])
      errores=extraerValorNumerico(ensayoActual[8])
      maxAciertosSeguidos=extraerValorNumerico(ensayoActual[13])
    else:
      aciertos=extraerValorNumerico(ensayoActual[0])
      tiempoTotalImpulsivas=extraerValorNumerico(ensayoActual[2])
      omisiones=extraerValorNumerico(ensayoActual[3])
      categorias=extraerValorNumerico(ensayoActual[5])
      latenciaPromedio=extraerValorNumerico(ensayoActual[6])
      errores=extraerValorNumerico(ensayoActual[7])
      maxAciertosSeguidos=extraerValorNumerico(ensayoActual[12])
    
    n = aciertos+omisiones+errores
    aciertoErrorOmision = 0 if aciertos != ultimoEnsayo['Aciertos acomulados'] else 1 if errores != ultimoEnsayo['Errores acomulados'] else 2

    # Dado que solo se muestran las laencias luego del primer acierto, pero se suma a la latencia cuando hay errores y aciertos,
    # si hay errores pero solo un acierto, la latencia de este acierto debe tomarse con cuidado
    if aciertoErrorOmision == 2:
      latencia=""
    else:
      latencia = promedioInverso(latenciaPromedio, ultimoEnsayo['Latencia promedio'], aciertos,tipoPrueba,aciertoErrorOmision==0)
      # promedioInverso(latenciaPromedio, ultimoEnsayo['Latencia promedio'], aciertos) if aciertoErrorOmision < 2 else 0
      if metioNariz >= 0:
        latencia = metioNariz
      elif (aciertos <= 1 and errores > 0) or (n - ultimoEnsayo["No. ensayo"]) != 1:
        # Este es el primer acierto luego de algunos errores y no hay "metio la nariz a los X ms"
        latencia = ""

    nuevoEnsayo = {
      "No. ensayo": n,
      "Fue acierto, error u omision":aciertoErrorOmision,
      "%Aciertos":aciertos/n,
      "%Errores":errores/n,
      "%Omisiones":omisiones/n,
      "Latencia de respuesta":latencia,
      "Tiempo en respuestas impulsivas":tiempoTotalImpulsivas - ultimoEnsayo['Tiempo total impulsivas'],
      "Categorías":categorias,
      "Máximo aciertos seguidos":maxAciertosSeguidos,
      "Aciertos acomulados":aciertos,
      "Errores acomulados":errores,
      "Omisiones acomuladas":omisiones,
      "Tiempo total impulsivas":tiempoTotalImpulsivas,
      "Latencia promedio":latenciaPromedio
      }
    
  return nuevoEnsayo

### Diccionarios bloque

In [53]:
def diccionariosBloque(tipoPrueba):
    if tipoPrueba == 'HAB':
        dataDict={
        "Dirección": [],
        "Día": [],
        "Bloque": [],
        "No. ensayo": [],
        "Fue acierto":[],
        "Fue acierto derecho":[],
        "%Aciertos":[],
        "%Omisiones":[],
        "Latencia de respuesta (ms)":[],
        "Tiempo en respuestas impulsivas (s)":[],
        "Categorías":[],
        "Máximo aciertos seguidos":[],
        "Tiempo total del bloque (s)":[],
        }

        ultimoEnsayo={
        "No. ensayo": 0,
        "Fue acierto":-1,
        "Fue acierto derecho":-1,
        "%Aciertos":-1,
        "%Omisiones":-1,
        "Latencia de respuesta":-1,
        "Tiempo en respuestas impulsivas":-1,
        "Categorías":-1,
        "Máximo aciertos seguidos":-1,
        "Aciertos acomulados":0,
        "Tiempo total impulsivas":0,
        "Total correctas derecha":0,
        "Total correctas izquierda":0,
        "Latencia promedio":0
        }

    elif tipoPrueba.find('SM')>-1:
        dataDict={
            "Dirección": [],
            "Día": [],
            "Bloque": [],
            "Fase": [],
            "Lado": [],
            "No. ensayo": [],
            "Acierto/Error/Omision":[],
            "%Aciertos":[],
            "%Errores":[],
            "%Omisiones":[],
            "Latencia de respuesta (ms)":[],
            "Tiempo en respuestas impulsivas (s)":[],
            "Categorías":[],
            "Máximo aciertos seguidos":[],
            "Tiempo total del bloque (s)":[],
            }

        ultimoEnsayo={
            "No. ensayo": -1,
            "Fue acierto, error u omision":-1,
            "%Aciertos":-1,
            "%Errores":-1,
            "%Omisiones":-1,
            "Latencia de respuesta":-1,
            "Tiempo en respuestas impulsivas":-1,
            "Categorías":-1,
            "Máximo aciertos seguidos":-1,
            "Aciertos acomulados":0,
            "Errores acomulados":0,
            "Omisiones acomuladas":0,
            "Tiempo total impulsivas":0,
            "Latencia promedio":0
            }
    return dataDict, ultimoEnsayo
    

### Extraer datos bloque

In [54]:
def extraerDatosBloque(texto, esCSV:bool, fileName, diaEnsayo, bloque, idAnimal, tipoPrueba, etiqueta):
    """
    Se carga el registro de cada animal en formato de texto y se realiza la 
    extracción de datos
    
    Parameters:
    texto (Array of String): Arreglo donde cada entrada es una línea del texto
    fileName (String): dirección del archivo de registro
    diaEnsayo (Int): día del ensyo en que se tomaron los datos
    bloque (Int): bloque dentro del día en que se tomaron los datos
    tipoPrueba (String): Cadena de texto con el nombre de la prueba
    etiqueta (String): Indica si hay más de un registro en el archivo "SPLIT" o si no tiene el tiempo "TIEMPO"

    Returns:
    dataDict (Pandas Dataframe): diccionario con los datos del ensayo
    """

    dataDict, ultimoEnsayo = diccionariosBloque(tipoPrueba)
    # Variables para registrar el tiempo
    hayTimestamps = etiqueta != "TIEMPO"
    horaInicio=""
    horaFin=""
    
    ensayoActual = []
    lineasSinTiempo = 0
    latenciasNegativas = 0

    if not esCSV:
      # Se eliminan las líneas iniciales
      texto = quitarEncabezadoNoCSV(texto)

    logging=False
    
    metioNariz=-1
    for linea in texto:
      # Cada línea se espera que tenga la siguiente estructura "12:20:12.886 -> Transmitiendo"
      hora, registro = separarLinea(linea)

      # Registra la primera hora del documento y luego, la reemplaza por el inicio de los ensayos, si lo hay
      if hayTimestamps:
        if hora == -1:
          lineasSinTiempo+=1
        else:
          if registro.find("Van a empezar los 50 ensayos")>-1 or horaInicio=="":
            horaInicio=datetime.strptime(hora.split('.')[0],"%H:%M:%S")
          horaFin=datetime.strptime(hora.split('.')[0],"%H:%M:%S")
      if registro.find("Terminan bloque de 50 ensayos")>-1:
        break
      
      # Cada línea se lee y procesa dependiendo del tipo de prueba y de si está separado por comas (CSV)
      procesarLinea = False

      # "Metio la nariz en la izquierda/derecha a los:"
      if "a los:" in registro:
        metioNariz = extraerValorNumerico(registro)

      if esCSV and registro.find('Ensayo')==-1:
        ensayoActual = registro.split(',')
        if (tipoPrueba == 'HAB' and len(ensayoActual) == 11) or (tipoPrueba.find('SM') > -1 and len(ensayoActual) == 14):
          ultimoEnsayo = procesarEnsayo(ensayoActual, esCSV, ultimoEnsayo, tipoPrueba, metioNariz)
          procesarLinea = True
      else:
        if registro.find('Aciertos:')>-1:
          logging=True
        elif logging and registro.find('--------------------------------------------------')>-1:
          logging=False
          procesarLinea=True
          ultimoEnsayo = procesarEnsayo(ensayoActual, esCSV, ultimoEnsayo, tipoPrueba, metioNariz)
          ensayoActual = []
        if logging:
          ensayoActual.append(registro)

      if procesarLinea:
        if tipoPrueba == 'HAB':
          dataDict["Dirección"].append(fileName)
          dataDict["Día"].append(diaEnsayo)
          dataDict["Bloque"].append(bloque)
          dataDict["No. ensayo"].append(ultimoEnsayo["No. ensayo"])
          dataDict["Fue acierto"].append(ultimoEnsayo["Fue acierto"])
          dataDict["Fue acierto derecho"].append(ultimoEnsayo["Fue acierto derecho"])
          dataDict["%Aciertos"].append(ultimoEnsayo["%Aciertos"])
          dataDict["%Omisiones"].append(ultimoEnsayo["%Omisiones"])
          dataDict["Latencia de respuesta (ms)"].append(ultimoEnsayo["Latencia de respuesta"])
          dataDict["Tiempo en respuestas impulsivas (s)"].append(ultimoEnsayo["Tiempo en respuestas impulsivas"])
          dataDict["Categorías"].append(ultimoEnsayo["Categorías"])
          dataDict["Máximo aciertos seguidos"].append(ultimoEnsayo["Máximo aciertos seguidos"])
        elif tipoPrueba.find('SM') > -1:
          dataDict["Dirección"].append(fileName)
          dataDict["Día"].append(diaEnsayo)
          dataDict["Bloque"].append(bloque)
          dataDict["Fase"].append(tipoPrueba.split('_')[1])
          dataDict["Lado"].append(tipoPrueba.split('_')[2])
          dataDict["No. ensayo"].append(ultimoEnsayo["No. ensayo"])
          dataDict["Acierto/Error/Omision"].append(ultimoEnsayo["Fue acierto, error u omision"])
          dataDict["%Aciertos"].append(ultimoEnsayo["%Aciertos"])
          dataDict["%Errores"].append(ultimoEnsayo["%Errores"])
          dataDict["%Omisiones"].append(ultimoEnsayo["%Omisiones"])
          dataDict["Latencia de respuesta (ms)"].append(ultimoEnsayo["Latencia de respuesta"])
          dataDict["Tiempo en respuestas impulsivas (s)"].append(ultimoEnsayo["Tiempo en respuestas impulsivas"])
          dataDict["Categorías"].append(ultimoEnsayo["Categorías"])
          dataDict["Máximo aciertos seguidos"].append(ultimoEnsayo["Máximo aciertos seguidos"])
        
        if ultimoEnsayo['Latencia de respuesta']!="" and (ultimoEnsayo['Latencia de respuesta'] < -1 or ultimoEnsayo['Latencia de respuesta'] > 10000):
          latenciasNegativas+=1


    try:
      segundosTotales=(horaFin-horaInicio).total_seconds()
      if segundosTotales < 0:
          horaFin -= timedelta(hours=1)
          horaInicio -= timedelta(hours=1)
          segundosTotales=(horaFin-horaInicio).total_seconds()
      dataDict["Tiempo total del bloque (s)"] = [segundosTotales]*len(dataDict['Dirección'])
    except:
      dataDict["Tiempo total del bloque (s)"] = [-1]*len(dataDict['Dirección'])
      if hayTimestamps:
        print(fileName +", No se puede calcular el tiempo total del experimento\n")
          
    if ultimoEnsayo["No. ensayo"] != len(dataDict['Dirección']):
      print(f'El bloque {fileName} tiene {len(dataDict["Dirección"])} ensayos, pero termina en {ultimoEnsayo["No. ensayo"]}' )
    if lineasSinTiempo > 0:
      print(f'El archivo {fileName} tiene {lineasSinTiempo} línea sin marca de tiempo')
    if latenciasNegativas > 0:
      print(f'El archivo {fileName} tiene {latenciasNegativas} ensayos con latencias fuera de rango')
    return pd.DataFrame.from_dict(dataDict)

In [55]:
promedioInverso(14386, 16352, 1)

-1966

## 1. Extraer lista de los registros

Lo primero que se hará es extraer la lista de los archivos que hay disponibles. Para esto, se recorrerán de forma iterativa las carpetas donde están los datos almacenados

In [12]:
def listaDeRegistros(base):
    """
    Toma una ubicación y realiza un proceso iterativo buscando en todos los 
    archivos que se encuentren almacenados
    
    Parameters:
    base (str): dirección del directorio base donde se buscarán los archivos que
    haya presentes

    Returns:
    registros (Array of str): lista de registros presentes en la carpeta

   """
    registros=[]
    for i in os.listdir(base):
        currentDirection = os.path.join(base,i)
        if os.path.isdir(currentDirection):
            registros+=listaDeRegistros(currentDirection)
        elif os.path.isfile(currentDirection):
            registros.append(currentDirection)
    return registros

In [13]:
def listaDeAnimales(registros, sexChar):
    """
    Toma el nombre de archivo de un registro y la divide, extrayendo los datos
    correspondientes, asumiendo que cumple con la convención de formato
    [DD].[MM].[AAAA].[Sex]_C[#Caja]_[#Animal]_[IdPrueba]_D[#dia]_B[#Bloque].[optional].docx
    
    Parameters:
    registros (Array of str): lista de registros
    sexo (Char): Caracter que puede ser 'H' para hembras y 'M' para machos

    Returns:
    data (Pandas dataframe): dataframe de pandas con los datos de cada registro
   """
   
    dia=[]
    mes=[]
    año=[]
    sexo=[]
    caja=[]
    marca=[]
    tipoPrueba=[]
    diaEnsayo=[]
    bloque=[]
    nombreArchivo=[]
    etiqueta=[]
    
    for registro in registros:
        try:
            # Ejemplo de registro '3.12.2021.H_C21_5_REV_AZ_D8_B1.docx'
            direccion=registro.split(os.sep)[-1:][0].split(".")
            
            # '3' '12' '2021'
            dia.append(int(direccion[0]))
            mes.append(int(direccion[1]))
            año.append(int(direccion[2]))
            
            # 'H_C21_5_REV_AZ_D8_B1'
            metadata=[]
            for i in direccion[3].split('_'):
                i = i.strip()
                if ' ' in i.strip():
                    for j in i.split(' '):
                        if j!='':
                            metadata.append(j)
                else:
                    metadata.append(i)
                        
            
            if sexChar in metadata[0]:
                sexo.append(metadata.pop(0).strip())
            else:
                sexo.append(sexChar)
                # print("Error al leer el sexo del animal: "+registro)
                
            if 'C' in metadata[0]:
                caja.append(metadata.pop(0).strip()[1:])
            else:
                caja.append(-1)
                print("Error al leer el identificador de la caja: "+registro)
                
            try:
                int(metadata[0])
                marca.append(metadata.pop(0).strip())
            except:
                marca.append(-1)
                print("Error al leer el numero del animal: "+registro)
                
            if 'B' in metadata[-1]:
                bloque.append(metadata.pop().strip()[1:])
            else:
                bloque.append(-1)
                print("Error al leer el bloque: "+registro)
                
            if 'D' in metadata[-1]:
                diaEnsayo.append(metadata.pop().strip()[1:])
            else:
                diaEnsayo.append(-1)
                print("Error al leer el bloque: "+registro)
            
            tipo=""
            for i in metadata:
                tipo+=(i.strip()+'_')
            tipoPrueba.append(tipo.strip()[:-1])
            
            if len(direccion)==6:
                etiqueta.append(direccion[4].strip())
            elif re.search('ESTANDARI', registro, re.IGNORECASE):
                etiqueta.append('estandarizacion')
            else:
                etiqueta.append('')
            
            nombreArchivo.append(registro)
            
        except:
            print("Error adicional: "+registro)
    d={'dia':dia, 'mes':mes, 'año':año, 'sexo':sexo, 'caja':caja,
       'marca':marca, 'tipoPrueba':tipoPrueba, 'diaEnsayo':diaEnsayo,
       'bloque':bloque, "nombreArchivo":nombreArchivo, 'etiqueta':etiqueta}
    data=pd.DataFrame(data=d)
    return data

In [14]:
def estandarizarTerminosTipoDePrueba(data):
    """
    Se tienen variaciones del identificador usado para los 7 tipos de prueba
    que se realizaron. Este método unifica los diferentes términos usados
    
    Parameters:
    data (Pandas dataframe): dataframe de pandas con los datos de cada registro

    Returns:
    cleanData (Pandas dataframe): dataframe de pandas con los términos unificados para la columna tipoPrueba
    """
    cleanData=data.copy()
    replaceDict={"_HAB":"HAB",
                 "P_DER":"SM_ADQ_DER", "SM_P_DER":"SM_ADQ_DER",
                 "SM_PRE_DER":"SM_ADQ_DER", "SM_ADQ_DER_":"SM_ADQ_DER","SM_DER_ADQ":"SM_ADQ_DER",
                 "SMM_ADQ_DER":"SM_ADQ_DER",
                 "P_IZQ":"SM_ADQ_IZQ","SM_P_IZQ":"SM_ADQ_IZQ","SM_IZQ_ADQ":"SM_ADQ_IZQ",
                 "SM_IZQ":"SM_ADQ_IZQ","SM_PRE_IZQ":"SM_ADQ_IZQ",
                 
                 "R_DER":"SM_REV_DER","SM_R_DER":"SM_REV_DER",
                 "R_IZQ":"SM_REV_IZQ","SM_R_IZQ":"SM_REV_IZQ",
                 
                 "DIS":"RA_ADQ", "DIS_AZ":"RA_ADQ", "ADQ_AZ":"RA_ADQ",
                 
                 "REV_AZ":"RA_REV", "RA":"RA_REV", "REV":"RA_REV", "R_AZ":"RA_REV",
                 }
    cleanData['tipoPrueba'].replace(to_replace=replaceDict, inplace=True)
    return cleanData

In [15]:
registrosHembras=estandarizarTerminosTipoDePrueba(listaDeAnimales(listaDeRegistros(pathHembras), 'H'))
registrosHembras.head(2)

Error adicional: D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/HEMBRAS\ACTUALIZADA. REGISTRO APRENDIZAJE EXPERIMENTO -iMac de Diego.xlsx
Error adicional: D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/HEMBRAS\Cronograma Actividades Experimento.xlsx


,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,27,8,2021,H,51,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,27,8,2021,H,51,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,


In [16]:
registrosMachos=estandarizarTerminosTipoDePrueba(listaDeAnimales(listaDeRegistros(pathMachos), 'M'))
registrosMachos.head(2)

,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,13,8,2021,M,10,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,


In [17]:
registrosUnificados=pd.concat([registrosMachos, registrosHembras], ignore_index=True)
registrosUnificados.insert(loc=0, column='IdAnimal', value=registrosUnificados.apply(lambda fila: f"{fila.sexo}-{fila.caja}-{fila.marca}", axis=1))
replaceDict={"Split":"SPLIT",
             "split":"SPLIT",
             "TIME":"TIEMPO",
             "estantadarizacion":"estandarizacion"
                 }
registrosUnificados['etiqueta'].replace(to_replace=replaceDict, inplace=True)
registrosUnificados.to_excel("registrosMachosYHembras.xlsx")
registrosUnificados.head()

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,M-10-1,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,M-10-1,13,8,2021,M,10,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,M-10-2,13,8,2021,M,10,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,M-10-2,13,8,2021,M,10,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,M-12-1,13,8,2021,M,12,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,MD


### 1.1. Análisis de calidad de los registros

Revisaremos los valores de cada columna para identificar la calidad de los datos

In [18]:
for column in registrosUnificados:
    if column != 'nombreArchivo':
        print({column:registrosUnificados[column].unique()})

{'IdAnimal': array(['M-10-1', 'M-10-2', 'M-12-1', 'M-12-2', 'M-11-1', 'M-11-2',
       'M-11-3', 'M-11-4', 'M-5B6-1', 'M-5B6-4', 'M-5B6-5', 'M-14-1',
       'M-14-2', 'M-21-1', 'M-21-2', 'M-21-3', 'M-24-3', 'M-24-4',
       'M-25-1', 'M-25-3', 'M-25-4', 'M-27-3', 'M-27-4', 'M-31-3',
       'M-31-4', 'M-22-3', 'M-22-4', 'M-23-1', 'M-23-2', 'M-26-1',
       'M-26-2', 'M-28-1', 'M-28-5', 'M-33-4', 'M-33-5', 'M-34-1',
       'M-34-6', 'M-29-5', 'M-29-6', 'M-29-7', 'M-32-1', 'M-32-2',
       'M-32-3', 'M-30-1', 'M-30-2', 'M-30-3', 'M-35-2', 'M-35-4',
       'M-17-1', 'M-19-1', 'M-19-3', 'M-20-2', 'M-20-3', 'M-13-1',
       'M-13-2', 'M-15-1', 'M-15-2', 'M-16-1', 'M-16-2', 'M-17-4',
       'M-17-2', 'H-51-1', 'H-51-2', 'H-51-3', 'H-55-1', 'H-55-2',
       'H-55-3', 'H-56-4', 'H-56-5', 'H-56-6', 'H-56-7', 'H-68-1',
       'H-68-2', 'H-68-3', 'H-60-1', 'H-60-2', 'H-60-3', 'H-61-4',
       'H-61-5', 'H-61-6', 'H-70-1', 'H-70-2', 'H-70-3', 'H-71-4',
       'H-71-6', 'H-63-1', 'H-63-2', 'H-64-3',

In [19]:
for column in registrosMachos:
    if column != 'nombreArchivo':
        print({column:registrosMachos[column].unique()})

{'dia': array([13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
       31,  1, 10, 11, 12, 18,  2,  3,  4,  5,  6,  7,  8,  9],
      dtype=int64)}
{'mes': array([ 8, 12,  1,  2, 11, 10,  9], dtype=int64)}
{'año': array([2021, 2022], dtype=int64)}
{'sexo': array(['M'], dtype=object)}
{'caja': array(['10', '12', '11', '5B6', '14', '21', '24', '25', '27', '31', '22',
       '23', '26', '28', '33', '34', '29', '32', '30', '35', '17', '19',
       '20', '13', '15', '16'], dtype=object)}
{'marca': array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)}
{'tipoPrueba': array(['HAB', 'SM_ADQ_DER', 'SM_ADQ_IZQ', 'SM_REV_DER', 'SM_REV_IZQ',
       'RA_ADQ', 'RA_REV'], dtype=object)}
{'diaEnsayo': array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '17', '11',
       '19', '12', '13', '14', '15', '16', '18', '20', '21', '22', '23',
       '24'], dtype=object)}
{'bloque': array(['1', '2'], dtype=object)}
{'etiqueta': array(['', 'MD', 'TIEMPO', 'estandarizacion', 'estanta

In [20]:
for column in registrosHembras:
    if column != 'nombreArchivo':
        print({column:registrosHembras[column].unique()})

{'dia': array([27, 28, 29, 30, 31,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
       13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 26, 24, 15, 25],
      dtype=int64)}
{'mes': array([ 8, 12,  1, 11, 10,  9], dtype=int64)}
{'año': array([2021, 2022], dtype=int64)}
{'sexo': array(['H'], dtype=object)}
{'caja': array(['51', '55', '56', '68', '60', '61', '70', '71', '63', '64', '67',
       '66', '72', '73', '58', '59', '62', '69', '52', '57'], dtype=object)}
{'marca': array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)}
{'tipoPrueba': array(['HAB', 'SM_DIS_IZQ', 'SM_REV_DER', 'SM_DIS_DER', 'SM_REV_IZQ',
       'RA_REV', 'RA_ADQ', 'SM_ADQ_DER', 'SM_ADQ_IZQ'], dtype=object)}
{'diaEnsayo': array(['1', '2', '3', '4', '7', '8', '5', '17', '6', '11', '9', '18',
       '10', '12', '19', '13', '14', '15', '16', '20', '21', '22'],
      dtype=object)}
{'bloque': array(['1', '2'], dtype=object)}
{'etiqueta': array(['', 'TIME', 'SPLIT', 'MD', 'Weird'], dtype=object)}


In [21]:
registrosUnificados.query('etiqueta == "resumen_final"')['nombreArchivo'].values

array(['D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\\OCTUBRE\\10 oct\\10.10.2021. C17_1_HAB_D6_B1. resumen_final.docx'],
      dtype=object)

In [22]:
registrosUnificados.query('etiqueta == "estandarizacion"')['IdAnimal'].unique()

array(['M-5B6-1', 'M-5B6-4', 'M-5B6-5', 'M-14-1', 'M-14-2'], dtype=object)

In [23]:
registrosUnificados.query('caja == "14" & sexo=="M"')

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
288,M-14-1,31,8,2021,M,14,1,RA_REV,3,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
289,M-14-2,31,8,2021,M,14,2,RA_REV,7,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion


Los datos de la columna 'etiqueta' indican errores que se encontraron luego de una revisión manual

| Etiqueta | Descripción |
| --- | --- |
| MD | Datos perdidos, están en blanco |
| TIME/TIEMPO | No tienen el registro del tiempo |
| .resumen_final | Solo arrojaron el resumen de todo el bloque (ensayo 50) |
| SPLIT/Split/split | Archivos que tienen entre 30 y 50 ensayos, los cuales no son consecutivos |
| EXCLUDE |  Machos C35_2 Y C21_2. Uno de ellos no logró el criterio de habituación en 20 días y debimos sacarlo del estudio, mientras que el otro se murió por el camino. De ese animal solo tenemos los datos de habituación que pueden servir (C35_2) |
| error de asignación | Se excluyen dado que se hizo otro protocolo al que correspondía ese día |
| estandarizacion | Se exlcuyen del análisis |

### 1.2. Estadísticos generales
Se mirará el número de datos para cada tipo de prueba en machos y hembras

In [24]:
# Número de archivos por cada tipo de prueba en machos
registrosUnificados['tipoPrueba'].value_counts()

RA_REV        2471
HAB            824
RA_ADQ         614
SM_REV_DER     364
SM_ADQ_IZQ     223
SM_REV_IZQ     209
SM_ADQ_DER     150
SM_DIS_IZQ      31
SM_DIS_DER       6
Name: tipoPrueba, dtype: int64

In [25]:
idMachos = registrosUnificados.query('sexo == "M"')['IdAnimal'].unique()
filesPerAnimal = {}
for id in idMachos:
    fileCount = {}
    countByType = registrosUnificados.query(f'IdAnimal == "{id}" & (etiqueta == "" | etiqueta == "TIEMPO" | etiqueta == "SPLIT")')['tipoPrueba'].value_counts().rename_axis('unique_values').reset_index(name='counts')
    for fileType in countByType['unique_values'].unique():
        fileCount[fileType] = countByType['counts'][countByType['unique_values'] == fileType ].values[0]
    filesPerAnimal[id] = fileCount
pd.DataFrame.from_dict(filesPerAnimal, orient='index').to_excel('Número de archivos por tipo de prueba machos.xlsx')

In [26]:
idMachos = registrosUnificados.query('sexo == "H"')['IdAnimal'].unique()
filesPerAnimal = {}
for id in idMachos:
    fileCount = {}
    countByType = registrosUnificados.query(f'IdAnimal == "{id}" & (etiqueta == "" | etiqueta == "TIEMPO" | etiqueta == "SPLIT")')['tipoPrueba'].value_counts().rename_axis('unique_values').reset_index(name='counts')
    for fileType in countByType['unique_values'].unique():
        fileCount[fileType] = countByType['counts'][countByType['unique_values'] == fileType ].values[0]
    filesPerAnimal[id] = fileCount
pd.DataFrame.from_dict(filesPerAnimal, orient='index').to_excel('Número de archivos por tipo de prueba hembras.xlsx')

## 2. Extraer datos de cada tipo de prueba
Vamos a extraer los datos de cada tipo de prueba. Para esto definiremos el enabezado de cada tipo de prueba que vamos a revisar y leeremos cada archivo.

¡OJO! El máximo número de bloques a analizar es 14 por animal

### 2.1. Extraer datos habituación
Cada prueba de habituación cuenta con 50 ensayos. En cada uno de ellos se tienen los siguientes datos:

| Nombre columna | Descripción | Rango |
| --- | --- | --- |
| Ensayo | Número de ensayo | 1 - 50 |
| Aciertos | Número acomulado de aciertos | 0 - 50 |
| Porcentaje aciertos | | 0% - 100% |
| Tiempo total en respuestas impulsivas | Tiempo acomulado que pasa metiendo a la nariz entre ensayos | Continua (segudos) |
| Omisiones | Número acomulado de omisiones | 0 - 50 |
| Porcentaje omisiones | | 0% - 100% |
| Categorias | Cada vez que complete 3 aciertos seguidos se cuenta una categoría| 0 - 16 |
| Total correctas a la derecha | | 0 - 50 |
| Total correctas a la izquierda | | 0 - 50 |
| Latencia promedio | Tiempo promedio en ensayos con aciertos | Contínua (milisegundos) |
| Maximo de aciertos seguidos | | 0- 50 |

Algunos archivos codifican las respuestas impulsivas contándolas y no como el tiempo. Dada la sensibilidad del sensor, esto lleva en algunos casos a números muy elevados.

Se tomará un archivo de habituación y se realizará la extracción de datos. Luego se aplicará a todos los archivos y se revisarán los casos donde no pudo extraer los datos.

Se extraerán los siguientes datos de cada archivo

|dia|bloque|Ensayo|AciertoOmision|LadoAcierto|PorcentajeAciertos|PorcentajeOmisiones|LatenciaDeRespuesta|TiempoEnRespuestaImpulsiva|Categorías|MaxAciertosSeguidos|TiempoTotalDelBloque|
|---|---|---|---|---|---|---|---|---|---|---|---|

In [27]:
registrosUnificados.loc[registrosUnificados['tipoPrueba'] == 'HAB'].head(1)['nombreArchivo'].values

array(['D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\\AGOSTO\\13.08.2021\\13.08.2021. C10_1 HAB D1 B1.docx'],
      dtype=object)

In [72]:
texto,esCSV=leerTexto('D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/DICIEMBRE/16 dic/BLOQUE 4/16.12.2021. C33_4_HAB_D3_B1.docx')
extraerDatosBloque(texto,
                                                      esCSV,
                                                      'D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/DICIEMBRE/16 dic/BLOQUE 4/16.12.2021. C33_4_HAB_D3_B1.docx',
                                                      '1',
                                                      '1',
                                                      'M-10-1',
                                                      'HAB',
                                                      'TIEMPO'
                                                      ).head()

,Dirección,Día,Bloque,No. ensayo,Fue acierto,Fue acierto derecho,%Aciertos,%Omisiones,Latencia de respuesta (ms),Tiempo en respuestas impulsivas (s),Categorías,Máximo aciertos seguidos,Tiempo total del bloque (s)
0,D:/Users/USER/OneDrive - Universidad de los An...,1,1,1.0,False,,0.000000,1.000000,,1.0,0.0,0.0,-1
1,D:/Users/USER/OneDrive - Universidad de los An...,1,1,2.0,True,False,0.500000,0.500000,347.0,0.0,0.0,1.0,-1
2,D:/Users/USER/OneDrive - Universidad de los An...,1,1,3.0,True,True,0.666667,0.333333,3839.0,0.0,0.0,2.0,-1
3,D:/Users/USER/OneDrive - Universidad de los An...,1,1,4.0,True,True,0.750000,0.250000,7085.0,0.0,1.0,3.0,-1
4,D:/Users/USER/OneDrive - Universidad de los An...,1,1,5.0,True,False,0.800000,0.200000,157.0,0.0,1.0,4.0,-1


In [36]:
#Vamos a ver los datos que se tienen por cada animal
output=pd.DataFrame()
outputAnimal=pd.DataFrame()
animal='M-23-1'
for indice, fila in registrosUnificados[ registrosUnificados['IdAnimal']==animal].iterrows():        
    if not (fila['etiqueta'] == '' or fila['etiqueta'] == 'TIEMPO'):
        continue
    else:
        try:
            # Datos que queremos guardar de cada registro
            tipoPrueba=fila['tipoPrueba']
            sexo=fila['sexo']
            
            # Convertimos el .docx a texto
            if tipoPrueba == 'HAB':
                texto,esCSV=leerTexto(fila['nombreArchivo'])
                registroSesion=extraerDatosBloque(texto,
                                                    esCSV,
                                                    fila['nombreArchivo'],
                                                    fila['diaEnsayo'],
                                                    fila['bloque'],
                                                    fila['IdAnimal'],
                                                    tipoPrueba,
                                                    fila['etiqueta']
                                                    )
                output=pd.concat([output, registroSesion],  ignore_index=True)
        except:
            print(f"{fila['nombreArchivo']} Error al leer el archivo")
    
output.to_excel(os.path.join(pathProcesarDatos,'Habituación',f'{animal}_HAB.xlsx'))

El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\DICIEMBRE\12 dic\BLOQUE 3\12.12.2021. C23_1_HAB_D3_B1.docx tiene 40 ensayos, pero termina en 50.0
El archivo D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\DICIEMBRE\12 dic\BLOQUE 3\12.12.2021. C23_1_HAB_D3_B1.docx tiene 4 línea sin marca de tiempo


In [30]:
#Vamos a ver los datos que se tienen por cada animal
outputHabituacion=pd.DataFrame()
for animal in registrosUnificados['IdAnimal'].unique():
    output=pd.DataFrame()
    outputAnimal=pd.DataFrame()
    for indice, fila in registrosUnificados[ registrosUnificados['IdAnimal']==animal].iterrows():        
        if not (fila['etiqueta'] == '' or fila['etiqueta'] == 'TIEMPO'):
            continue
        else:
            try:
                # Datos que queremos guardar de cada registro
                tipoPrueba=fila['tipoPrueba']
                sexo=fila['sexo']
                
                # Convertimos el .docx a texto
                if tipoPrueba == 'HAB':
                    texto,esCSV=leerTexto(fila['nombreArchivo'])
                    registroSesion=extraerDatosBloque(texto,
                                                      esCSV,
                                                      fila['nombreArchivo'],
                                                      fila['diaEnsayo'],
                                                      fila['bloque'],
                                                      fila['IdAnimal'],
                                                      tipoPrueba,
                                                      fila['etiqueta']
                                                      )
                    output=pd.concat([output, registroSesion],  ignore_index=True)
            except:
                print(f"{fila['nombreArchivo']} Error al leer el archivo")
        
    output.to_excel(os.path.join(pathProcesarDatos,'Habituación',f'{animal}_HAB.xlsx'))


El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\13.08.2021\13.08.2021. C10_1 HAB D1 B1.docx tiene 38 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\14.08.2021\14.08.2021. C10_1 HAB D2 B1.docx tiene 38 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\17.08.2021\17.08.2021. C11_1 HAB D3 B2.docx tiene 49 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\17.08.2021\17.08.2021. C11_3 HAB D3 B1.docx tiene 49 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\16.08.2021\16.08.2021. C11_4 HAB D2 B1.docx tiene 48 ensayos, pero termina en 50.0
El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\DICIEMBRE\12 dic\BLOQU

### 2.2. Extraer datos series motoras
Cada prueba de series motorascuenta con 50 ensayos. En cada uno de ellos se tienen los siguientes datos:

| Nombre columna | Descripción | Rango |
| --- | --- | --- |


Algunos archivos codifican las respuestas impulsivas contándolas y no como el tiempo. Dada la sensibilidad del sensor, esto lleva en algunos casos a números muy elevados.

Se tomará un archivo de habituación y se realizará la extracción de datos. Luego se aplicará a todos los archivos y se revisarán los casos donde no pudo extraer los datos.

Se extraerán los siguientes datos de cada archivo

|dia|bloque|Fase|Lado|Ensayo|AciertoErrorOmision|LadoAcierto|PorcentajeAciertos|PorcentajeOmisiones|LatenciaDeRespuesta|TiempoEnRespuestaImpulsiva|Categorías|MaxAciertosSeguidos|TiempoTotalDelBloque|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|

In [60]:
registrosUnificados.loc[registrosUnificados['tipoPrueba'] == 'SM_ADQ_DER'].head(1)['nombreArchivo'].values

array(['D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\\AGOSTO\\16.08.2021\\16.08.2021. C10_1 SM P_DER D1 B1.docx'],
      dtype=object)

In [92]:
texto,esCSV=leerTexto('D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/16.08.2021/16.08.2021. C10_1 SM P_DER D1 B1.docx')
extraerDatosBloque(texto,
                                                      esCSV,
                                                      'D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/16.08.2021/16.08.2021. C10_1 SM P_DER D1 B1.docx',
                                                      '1',
                                                      '1',
                                                      'M-10-1',
                                                      'SM_ADQ_DER',
                                                      ''
                                                      ).head(10)

El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//MACHOS/AGOSTO/16.08.2021/16.08.2021. C10_1 SM P_DER D1 B1.docx tiene 34 ensayos, pero termina en 50.0


,Dirección,Día,Bloque,Fase,Lado,No. ensayo,Acierto/Error/Omision,%Aciertos,%Errores,%Omisiones,Latencia de respuesta (ms),Tiempo en respuestas impulsivas (s),Categorías,Máximo aciertos seguidos,Tiempo total del bloque (s)
0,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,1.0,1,0.000000,1.000000,0.0,6994.0,3.0,0.0,0.0,1030.0
1,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,2.0,1,0.000000,1.000000,0.0,116.0,7.0,0.0,0.0,1030.0
2,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,3.0,0,0.333333,0.666667,0.0,7726.0,5.0,0.0,1.0,1030.0
3,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,4.0,1,0.250000,0.750000,0.0,1516.0,6.0,0.0,1.0,1030.0
4,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,5.0,1,0.200000,0.800000,0.0,641.0,26.0,0.0,1.0,1030.0
5,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,6.0,1,0.166667,0.833333,0.0,3805.0,10.0,0.0,1.0,1030.0
6,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,7.0,0,0.285714,0.714286,0.0,4179.0,0.0,0.0,1.0,1030.0
7,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,8.0,1,0.250000,0.750000,0.0,516.0,34.0,0.0,1.0,1030.0
8,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,9.0,1,0.222222,0.777778,0.0,1003.0,11.0,0.0,1.0,1030.0
9,D:/Users/USER/OneDrive - Universidad de los An...,1,1,ADQ,DER,10.0,1,0.200000,0.800000,0.0,3241.0,3.0,0.0,1.0,1030.0


In [91]:
print(promedioInverso(16352,14836, 1, 'SM', False))
print(promedioInverso(16993, 16352, 1, 'SM', False))
print(promedioInverso(20798, 16993, 1, 'SM', False))
print(promedioInverso(12488, 20798, 2, 'SM', True))
print(promedioInverso(12746, 12488, 2, 'SM', False))
print(promedioInverso(13248,12746, 2, 'SM', False))

1516
641
3805
4178
516
1004


In [94]:
#Vamos a ver los datos que se tienen por cada animal
outputHabituacion=pd.DataFrame()
for animal in registrosUnificados['IdAnimal'].unique():
    output=pd.DataFrame()
    outputAnimal=pd.DataFrame()
    for indice, fila in registrosUnificados[ registrosUnificados['IdAnimal']==animal].iterrows():        
        if not (fila['etiqueta'] == '' or fila['etiqueta'] == 'TIEMPO'):
            continue
        else:
            try:
                # Datos que queremos guardar de cada registro
                tipoPrueba=fila['tipoPrueba']
                sexo=fila['sexo']
                
                # Convertimos el .docx a texto
                if tipoPrueba.find('SM')>-1:
                    texto,esCSV=leerTexto(fila['nombreArchivo'])
                    registroSesion=extraerDatosBloque(texto,
                                                      esCSV,
                                                      fila['nombreArchivo'],
                                                      fila['diaEnsayo'],
                                                      fila['bloque'],
                                                      fila['IdAnimal'],
                                                      tipoPrueba,
                                                      fila['etiqueta']
                                                      )
                    output=pd.concat([output, registroSesion],  ignore_index=True)
            except:
                print(f"{fila['nombreArchivo']} Error al leer el archivo")
        
    output.to_excel(os.path.join(pathProcesarDatos,'Series motoras',f'{animal}_SM.xlsx'))

El bloque D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\16.08.2021\16.08.2021. C10_1 SM P_DER D1 B1.docx tiene 34 ensayos, pero termina en 50.0
El archivo D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\20.08.2021\20.08.2021. C10_1 SM_P_DER D4 B1.docx tiene 14 línea sin marca de tiempo
El archivo D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\20.08.2021\20.08.2021. C10_1 SM_P_DER D4 B2.docx tiene 14 línea sin marca de tiempo
El archivo D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\21.08.2021\21.08.2021. C10_1 SM REV IZQ D5 B1.docx tiene 14 línea sin marca de tiempo
El archivo D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOSTO\21.08.2021\21.08.2021. C10_1 SM REV IZQ D5 B2.docx tiene 14 línea sin marca de tiempo
El archivo D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/MACHOS\AGOS